Her vil vi forsøke å trene et nettverk som skal generere og predikere tekst

In [24]:
from utils import onehot
from data_generators import text_to_training_data
from text_generation import *
from layers import *
from neural_network import NeuralNetwork
import numpy as np

In [25]:
d = 80
n_max = 50
p = 100
k = 25
L = 2

text =  open('Harrypotter.txt', 'r').read()
data,idx_to_text,text_to_idx, m = text_to_training_data(n_max,text,num_batches=20,batch_size=100)

print("We will train on %d batches of size %d" % (len(data['x_train']),len(data['x_train'][0])))
print("Each sequence has length %d" % n_max)

print("Example of a sequence (chars): \n")
print(''.join([idx_to_text[i] for i in data['x_train'][0][0]]))

print("\nExample of a sequence (idx): \n")
print(data['x_train'][0][0])

data has 439360 characters, 79 unique.
We will train on 20 batches of size 100
Each sequence has length 50
Example of a sequence (chars): 

Mr. and Mrs. Dursley, of number four, Privet Drive

Example of a sequence (idx): 

[37 69 11  2 52 65 55  2 37 69 70 11  2 28 72 69 70 63 56 76  9  2 66 57
  2 65 72 64 53 56 69  2 57 66 72 69  9  2 40 69 60 73 56 71  2 28 69 60
 73 56]


In [26]:
embed = EmbedPosition(n_max,m,d)
att1 = Attention(d,k)
att2 = Attention(d,k)
att3 = Attention(d,k)
att4 = Attention(d,k)


ff1 = FeedForward(d,p)
ff2 = FeedForward(d,p)
ff3 = FeedForward(d,p)
ff4 = FeedForward(d,p)

un_embed = LinearLayer(d,m)
softmax = Softmax()

layers = [embed, att1, ff1, att2, ff2, att3, ff3, att4, ff4, un_embed, softmax]

net = NeuralNetwork(layers)
loss = CrossEntropy()

In [27]:
n_iter = 100
batches = 20

x_data = data['x_train']
y_data = data['y_train']


for n in range(n_iter):
    losses = []
    for b in range(batches):
        x = x_data[b]
        y = y_data[b]

        X = onehot(x,m)
        Z = net.forward(X)
        losses.append(loss.forward(Z,y))
        dLdZ = loss.backward()
        net.backward(dLdZ)
        net.step_Adam(0.001)
    print(np.mean(losses))

KeyboardInterrupt: 

In [ ]:
#We can now generate text from an initial string
start_text = "He was a boy, his name was "
start_idx = np.array([convertToID(start_text,text_to_idx)])
print(start_idx)

[[32 56  2 74 52 70  2 52  2 53 66 76  9  2 59 60 70  2 65 52 64 56  2 74
  52 70  2]]


In [ ]:
#length of the total text sequence we want to generate
n_gen = n_max*20

generated_idx = generate(net,start_idx,m,n_max,n_gen)

text = convertToTxt(generated_idx[0],idx_to_text)

print(text)

He was a boy, his name was for their flutsidessible them window, not who was I have Frock be and a baby... had witch they behiling
on.. the mage wen --. On umba
-- Maday and a durage -- Alle. CarBus -- a dours just beake they were did. "Pamers -- Dumbledore people he was o' front" Harry said... Ck. Aunt Petunia had jumbling. "Eve
was whout the magic..... whis
hall."

Uncle Vernon raforelight he pating in
lapth. They he and Piers you'd jums bizards: Art f-Petter Muggle -- "G 2) ogli..... read put.. went , wasned he -"
pro, gluts -- but I want know. For, Prive. Drive, Putil whing. T-'s miss you drokess
grab -- you could ser."

Mr. Durs. Dursley It's he jumbled this, fast she
ceing snake
they know stare'cingrage wh
prop -"

Harry snapped the told, Seew-liied serangers ceh" said Vol-- car. Aunt Petonts she still, boy.

Top."

"But the shake roak! You was jushed. Herry shounside hughis obo. You was than, just
the disawould really they snak. The
shapped the man had be a shake hands Potters -